In [1]:
from pickhardtpayments.ChannelGraph import ChannelGraph
from pickhardtpayments.UncertaintyNetwork import UncertaintyNetwork
from pickhardtpayments.OracleLightningNetwork import OracleLightningNetwork
from pickhardtpayments.SyncSimulatedPaymentSession import SyncSimulatedPaymentSession

import ndjson

In [2]:
channel_graph = ChannelGraph("../channels.sample.json")
uncertainty_network = UncertaintyNetwork(channel_graph)
oracle_lightning_network = OracleLightningNetwork(channel_graph)
oracle_lightning_network_prior = OracleLightningNetwork(channel_graph)

In [3]:
source_pre = {}
for channel in oracle_lightning_network.network.edges:
    if channel[0] not in source_pre.keys():
        source_pre[channel[0]] = oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
    else:
        source_pre[channel[0]] += oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
print(source_pre)

{'A': 207500.0, 'B': 257500.0, 'C': 155000.0, 'D': 205000.0}


In [4]:
payment_set = ndjson.load(open("./same_payments_small_graph.ndjson", "r"))

In [5]:
# init simulation
c = 0
successful_payments = 0
failed_payments = 0
payment_simulation = []

# create new payment session - will later be moved before payment loop to simulate sequence of payments
sim_session = SyncSimulatedPaymentSession(oracle_lightning_network, uncertainty_network, prune_network=False)

In [6]:
for payment in payment_set:
    c += 1
    # create new payment session
    # sim_session = SyncSimulatedPaymentSession(oracle_lightning_network, uncertainty_network, prune_network=False)
    # we need to make sure we forget all learnt information on the Uncertainty Network
    # sim_session.forget_information()
    ret = sim_session.pickhardt_pay(payment["sender"], payment["receiver"], payment["amount"],
                                    mu=0, base=0, loglevel="debug")
    if ret > 0:
        successful_payments += 1
    if ret < 0:
        failed_payments += 1
    payment['success'] = ret
    payment_simulation.append(payment)

16:27:20.729 | INFO | *** new pickhardt payment ***
16:27:20.731 | INFO | log level is 10
16:27:20.736 | WARNING | placed 0 as min and 245000 as max liquidity, while 60000 is inflight
16:27:20.736 | WARNING | return_channel.min_liquidity 60000, return_channel.max_liquidity 305000
16:27:20.741 | INFO | Executing Payment...
16:27:20.743 | WARNING | inflights removed from uncertainty channel - 60000 sats.
16:27:20.745 | INFO | SUMMARY:
16:27:20.746 | INFO | ========
16:27:20.747 | INFO | Rounds of mcf-computations:	  1
16:27:20.748 | INFO | Number of attempts made:		  1
16:27:20.749 | INFO | Number of failed attempts:	  0
16:27:20.750 | INFO | Failure rate: 0.00% 
16:27:20.751 | INFO | total Payment lifetime (including inefficient memory management): 0.011 sec
16:27:20.751 | INFO | Learnt entropy:  0.63 bits
16:27:20.753 | INFO | fee for settlement of delivery:    0.000 sat --> 0 ppm
16:27:20.754 | INFO | used mu: 0
16:27:20.755 | INFO | Payment was successful: True
16:27:20.756 | INFO | 

In [7]:
print(f"\n{c} payments. {successful_payments} successful, {failed_payments} failed.")


12 payments. 3 successful, 9 failed.


In [8]:
source_post = {}
for channel in oracle_lightning_network.network.edges:
    if channel[0] not in source_post.keys():
        source_post[channel[0]] = oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
    else:
        source_post[channel[0]] += oracle_lightning_network.get_channel(channel[0], channel[1], channel[2]).actual_liquidity
print(source_post)

{'A': 207500.0, 'B': 407500.0, 'C': 155000.0, 'D': 55000.0}


## Display change in outbound liquidity

In [9]:
for key in source_pre:
    print(key, source_post[key] - source_pre[key])

A 0.0
B 150000.0
C 0.0
D -150000.0


## Display channel capacities and their change from before and after as well
## as intervals in Uncertainty Network

In [10]:
for edge in oracle_lightning_network.network.edges:
    channel_init = oracle_lightning_network_prior.get_channel(edge[0], edge[1], edge[2])
    channel_now = oracle_lightning_network.get_channel(edge[0], edge[1], edge[2])
    uncertainty_channel_now = uncertainty_network.get_channel(edge[0], edge[1], edge[2])
    print(f"{edge[0]}-{edge[1]} (capacity {channel_init.capacity}): {channel_init.actual_liquidity:>7.0f} to {channel_now.actual_liquidity:>7.0f}. "
          f"Intervall [{uncertainty_channel_now.min_liquidity:>7}; {uncertainty_channel_now.max_liquidity:>7}]. "
          f"oracle i_f: {channel_now.in_flight:>7}, uncertainty i_f {uncertainty_channel_now.in_flight:>7}")

A-B (capacity 105000):   52500 to   52500. Intervall [      0;  105000]. oracle i_f:       0, uncertainty i_f       0
A-C (capacity 205000):  102500 to  102500. Intervall [      0;  205000]. oracle i_f:       0, uncertainty i_f       0
A-D (capacity 105000):   52500 to   52500. Intervall [      0;  105000]. oracle i_f:       0, uncertainty i_f       0
B-A (capacity 105000):   52500 to   52500. Intervall [      0;  105000]. oracle i_f:       0, uncertainty i_f       0
B-C (capacity 105000):   52500 to   52500. Intervall [      0;  105000]. oracle i_f:       0, uncertainty i_f       0
B-D (capacity 305000):  152500 to  302500. Intervall [ 150000;  305000]. oracle i_f:       0, uncertainty i_f       0
C-B (capacity 105000):   52500 to   52500. Intervall [      0;  105000]. oracle i_f:       0, uncertainty i_f       0
C-A (capacity 205000):  102500 to  102500. Intervall [      0;  205000]. oracle i_f:       0, uncertainty i_f       0
D-A (capacity 105000):   52500 to   52500. Intervall [  

## is final liquidity of channel within range of uncertainty network estimate?

In [11]:
for edge in oracle_lightning_network.network.edges:
    channel_init = oracle_lightning_network_prior.get_channel(edge[0], edge[1], edge[2])
    channel_now = oracle_lightning_network.get_channel(edge[0], edge[1], edge[2])
    uncertainty_channel_now = uncertainty_network.get_channel(edge[0], edge[1], edge[2])
    if channel_now.actual_liquidity not in range(uncertainty_channel_now.min_liquidity, uncertainty_channel_now.max_liquidity):
        print(f"PROBLEM in channel {edge[0]}-{edge[1]}. liq: {channel_now.actual_liquidity}, min: {uncertainty_channel_now.min_liquidity}, max: {uncertainty_channel_now.max_liquidity}")
    else:
        print(f"channel {edge[0]}-{edge[1]} is ok")

channel A-B is ok
channel A-C is ok
channel A-D is ok
channel B-A is ok
channel B-C is ok
channel B-D is ok
channel C-B is ok
channel C-A is ok
channel D-A is ok
channel D-B is ok
